In [ ]:
!pip install datasets rouge-score bert-score transformers sentencepiece

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=81eb620c990c8dfcb2c4ae91aff13ef0f11a3acbe9c01cd24f338f7060a94b7d
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:
import torch
from datasets import load_dataset
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from rouge_score import rouge_scorer
from bert_score import score

In [ ]:
# 1. Load dataset (1000 samples)
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:1000]")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [5]:
# 2. Load PEGASUS model + tokenizer
model_name = "google/pegasus-cnn_dailymail"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def pegasus_summarize(text):
    inputs = tokenizer(text, truncation=True, padding="longest", return_tensors="pt", max_length=1024).to(model.device)
    summary_ids = model.generate(inputs["input_ids"], num_beams=5, max_length=128, min_length=30, length_penalty=2.0)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
# 3. Generate summaries
references, predictions = [], []
for sample in dataset:
    article = sample["article"]
    ref_summary = sample["highlights"]
    pred_summary = pegasus_summarize(article)

    references.append(ref_summary)
    predictions.append(pred_summary)

In [ ]:
# 4. Evaluation: ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
rouge1, rouge2, rougel = 0, 0, 0
for ref, pred in zip(references, predictions):
    scores = scorer.score(ref, pred)
    rouge1 += scores['rouge1'].fmeasure
    rouge2 += scores['rouge2'].fmeasure
    rougel += scores['rougeL'].fmeasure

In [ ]:
n = len(predictions)
print("ROUGE-1:", rouge1/n)
print("ROUGE-2:", rouge2/n)
print("ROUGE-L:", rougel/n)

In [ ]:
# 5. Evaluation: BERTScore
P, R, F1 = score(predictions, references, lang="en", verbose=True)
print("BERTScore F1:", F1.mean().item())

In [ ]:
 6. Show sample outputs
for i in range(3):
    print("\nARTICLE:", dataset[i]["article"][:400], "...")
    print("REFERENCE SUMMARY:", dataset[i]["highlights"])
    print("PEGASUS SUMMARY:", predictions[i])